# Prompting with Prompt Templates for LLM Input / Output with LangChain

## Install OpenAI, HuggingFace and LangChain dependencies

In [1]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

  Using cached langsmith-0.2.11-py3-none-any.whl.metadata (14 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.63-py3-none-any.whl.metadata (5.8 kB)
Using cached langchain_core-0.3.63-py3-none-any.whl (438 kB)
Using cached langsmith-0.2.11-py3-none-any.whl (326 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.4.2
    Uninstalling langsmith-0.4.2:
      Successfully uninstalled langsmith-0.4.2
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain-core]m [langchain-core]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depende

## Enter API Tokens

#### Enter your Open AI Key here

You can get the key from [here](https://platform.openai.com/api-keys) after creating an account or signing in

In [2]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

In [3]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Chat Models and LLMs

Large Language Models (LLMs) are a core component of LangChain. LangChain does not implement or build its own LLMs. It provides a standard API for interacting with almost every LLM out there.

There are lots of LLM providers (OpenAI, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

## Accessing Commercial LLMs like ChatGPT

In [4]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Prompt Templates
Prompt templates are pre-designed formats used to generate prompts for language models. These templates can include instructions, few-shot examples, and specific contexts and questions suited for particular tasks.

LangChain provides tools for creating and using prompt templates. It aims to develop model-agnostic templates to facilitate the reuse of existing templates across different language models. Typically, these models expect prompts in the form of either a string or a list of chat messages.

### Types of Prompt Templates

- **PromptTemplate:**
  - Used for creating string-based prompts.
  - Utilizes Python's `str.format` syntax for templating, supporting any number of variables, including scenarios with no variables.

- **ChatPromptTemplate:**
  - Designed for chat models, where the prompt consists of a list of chat messages.
  - Each chat message includes content and a role parameter. For instance, in the OpenAI Chat Completions API, a chat message could be assigned to an AI assistant, a human, or a system role.
- **FewShotChatMessagePromptTemplate**
  - A few-shot prompt template can be constructed from a set of examples


### PromptTemplate

We can use `PromptTemplate` to create a template for a string prompt.

By default, `PromptTemplate` uses Python's `str.format` syntax for templating.

You can create custom prompt templates that format the prompt in any way you want. For more information, see [Prompt Template Composition](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/composition/).

In [5]:
from langchain.prompts import PromptTemplate

# Simple prompt

prompt = """Explain to me what is Generative AI in 3 bullet points?"""
prompt_template = PromptTemplate.from_template(prompt)
prompt_template

PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Explain to me what is Generative AI in 3 bullet points?')

In [ ]:
print(prompt)

input_variables=['prompt_txt'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nAct as a Artificial Intelligence Expert.\nTransform the input research paper abstract given below\nbased on the instruction input by the user.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['prompt_txt'], input_types={}, partial_variables={}, template='{prompt_txt}'), additional_kwargs={})]


In [7]:
print(prompt_template)

input_variables=[] input_types={} partial_variables={} template='Explain to me what is Generative AI in 3 bullet points?'


In [8]:
prompt_template.format()

'Explain to me what is Generative AI in 3 bullet points?'

In [10]:
prompt_template.template

'Explain to me what is Generative AI in 3 bullet points?'

In [9]:
response = chatgpt.invoke(prompt_template.format())
print(response.content)

- **Definition**: Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or videos, by learning patterns and structures from existing data.

- **Techniques**: It often employs advanced techniques like deep learning, particularly neural networks such as Generative Adversarial Networks (GANs) and transformer models, to generate high-quality and coherent outputs.

- **Applications**: Generative AI is used in various fields, including creative arts (e.g., art and music generation), content creation (e.g., writing and journalism), and even in practical applications like drug discovery and personalized marketing.


In [11]:
response = chatgpt.invoke(prompt_template.template)
print(response.content)

- **Definition**: Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or videos, by learning patterns and structures from existing data.

- **Techniques**: It often employs advanced techniques like deep learning, particularly neural networks such as Generative Adversarial Networks (GANs) and transformer models, to generate high-quality and coherent outputs.

- **Applications**: Generative AI is used in various fields, including creative arts (e.g., art and music generation), content creation (e.g., writing and journalism), and even in practical applications like drug discovery and data augmentation.


In [13]:
# more complex prompt with placeholders
prompt = """Explain to me briefly about {topic} in {language}."""

prompt_template = PromptTemplate.from_template(prompt)
prompt_template

PromptTemplate(input_variables=['language', 'topic'], input_types={}, partial_variables={}, template='Explain to me briefly about {topic} in {language}.')

In [14]:
print(prompt_template)

input_variables=['language', 'topic'] input_types={} partial_variables={} template='Explain to me briefly about {topic} in {language}.'


In [19]:
inputs = [("Generative AI", "english"),
          ("Artificial Intelligence", "bengali"),
          ("Deep Learning", "german")]

prompts = [prompt_template.format(topic=topic, language=language) for topic, language in inputs]
prompts

['Explain to me briefly about Generative AI in english.',
 'Explain to me briefly about Artificial Intelligence in bengali.',
 'Explain to me briefly about Deep Learning in german.']

In [20]:
# use map to run on multiple prompts in one go
responses = chatgpt.map().invoke(prompts)

In [21]:
responses

[AIMessage(content='Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or even videos, based on the patterns and information they have learned from existing data. These models, like GPT-3 for text or DALL-E for images, use techniques such as deep learning and neural networks to generate outputs that can mimic human creativity. Generative AI has applications in various fields, including art, entertainment, marketing, and more, enabling users to produce unique and innovative content efficiently.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 18, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', '

In [22]:
for response in responses:
  print(response.content)
  print('-----')

Generative AI refers to a class of artificial intelligence models that can create new content, such as text, images, music, or even videos, based on the patterns and information they have learned from existing data. These models, like GPT-3 for text or DALL-E for images, use techniques such as deep learning and neural networks to generate outputs that can mimic human creativity. Generative AI has applications in various fields, including art, entertainment, marketing, and more, enabling users to produce unique and innovative content efficiently.
-----
কৃত্রিম বুদ্ধিমত্তা (Artificial Intelligence বা AI) হল একটি প্রযুক্তি যা কম্পিউটার এবং মেশিনকে মানুষের মতো চিন্তা, শেখা এবং সিদ্ধান্ত নিতে সক্ষম করে। এটি বিভিন্ন অ্যালগরিদম এবং ডেটা ব্যবহার করে কাজ করে, যার মাধ্যমে মেশিনগুলি তথ্য বিশ্লেষণ করে এবং সমস্যার সমাধান করতে পারে। কৃত্রিম বুদ্ধিমত্তা বিভিন্ন ক্ষেত্রে ব্যবহৃত হয়, যেমন স্বয়ংক্রিয় গাড়ি, স্বাস্থ্যসেবা, গ্রাহক সেবা, এবং আরও অনেক কিছু। AI-এর মূল উদ্দেশ্য হল মানুষের কাজকে সহজ করা এবং

### ChatPromptTemplate

The standard prompt format to [chat models](https://python.langchain.com/v0.1/docs/modules/model_io/chat/) is a list of [chat messages](https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/).

Each chat message is associated with content, and an additional parameter called `role`. For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

In [23]:
from langchain_core.prompts import ChatPromptTemplate

# simple prompt with placeholders
prompt = """Explain to me briefly about {topic}."""

chat_template = ChatPromptTemplate.from_template(prompt)
chat_template

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Explain to me briefly about {topic}.'), additional_kwargs={})])

In [39]:
from langchain.prompts import ChatPromptTemplate

# Simple template with variables
simple_template = ChatPromptTemplate.from_template(
    "Tell me a {adjective} joke about {topic}"
)

# Format the template with values
formatted_prompt = simple_template.format_messages(
    adjective="funny",
    topic="databases"
)

print(formatted_prompt)
# Output: [HumanMessage(content='Tell me a funny joke about databases')]

[HumanMessage(content='Tell me a funny joke about databases', additional_kwargs={}, response_metadata={})]


In [41]:
formatted_prompt

[HumanMessage(content='Tell me a funny joke about databases', additional_kwargs={}, response_metadata={})]

In [40]:
print(chatgpt.invoke(formatted_prompt))

content='Why did the database administrator break up with their partner?\n\nBecause they had too many "NULL" values in the relationship!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 14, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None} id='run--1b0be8df-0dbb-4402-9c44-4e741a0f486c-0' usage_metadata={'input_tokens': 14, 'output_tokens': 24, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [34]:
from langchain.prompts import ChatPromptTemplate

# Simple template with variables
simple_template = ChatPromptTemplate.from_template(
    "Tell me a {adjective} joke about {topic}"
)

# Format the template with values
formatted_prompt = simple_template.format(
    adjective="funny",
    topic="databases"
)

print(formatted_prompt)
# Output: [HumanMessage(content='Tell me a funny joke about databases')]

Human: Tell me a funny joke about databases


In [37]:
print(chatgpt.invoke(formatted_prompt))

content='Why did the database administrator break up with their partner?\n\nBecause they had too many "NULL" values in the relationship!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 16, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None} id='run--8c24450e-7f13-4fc0-81e2-b5d1ceecaf87-0' usage_metadata={'input_tokens': 16, 'output_tokens': 24, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [24]:
topics = ['mortgage', 'fractional real estate', 'commercial real estate']
prompts = [chat_template.format(topic=topic) for topic in topics]
prompts

['Human: Explain to me briefly about mortgage.',
 'Human: Explain to me briefly about fractional real estate.',
 'Human: Explain to me briefly about commercial real estate.']

In [25]:
responses = chatgpt.map().invoke(prompts)
for response in responses:
  print(response.content)
  print('-----')

A mortgage is a type of loan specifically used to purchase real estate. In a mortgage agreement, the borrower receives funds from a lender to buy a home or property and agrees to repay the loan over a specified period, typically 15 to 30 years. The property itself serves as collateral, meaning if the borrower fails to make payments, the lender can take possession of the property through a process called foreclosure. Mortgages usually involve monthly payments that include both principal (the amount borrowed) and interest (the cost of borrowing). There are various types of mortgages, including fixed-rate and adjustable-rate mortgages, each with different terms and conditions.
-----
Fractional real estate refers to a shared ownership model where multiple investors collectively own a property, allowing them to invest in real estate without having to purchase an entire property individually. Each investor holds a fraction of the property, which entitles them to a proportional share of the i

In [0]:
responses[0]

In [26]:
# more complex prompt with a series of messages
messages = [
        ("system", "Act as an expert in real estate and provide brief answers"),
        ("human", "what is your name?"),
        ("ai", "my name is AIBot"),
        ("human", "{user_prompt}"),
]
chat_template = ChatPromptTemplate.from_messages(messages)
chat_template

ChatPromptTemplate(input_variables=['user_prompt'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Act as an expert in real estate and provide brief answers'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='what is your name?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='my name is AIBot'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_prompt'], input_types={}, partial_variables={}, template='{user_prompt}'), additional_kwargs={})])

In [27]:
text_prompts = ["what is your name?",
                "explain commercial real estate to me"]
chat_prompts = [chat_template.format(user_prompt=prompt) for prompt in text_prompts]
chat_prompts

['System: Act as an expert in real estate and provide brief answers\nHuman: what is your name?\nAI: my name is AIBot\nHuman: what is your name?',
 'System: Act as an expert in real estate and provide brief answers\nHuman: what is your name?\nAI: my name is AIBot\nHuman: explain commercial real estate to me']

In [28]:
print(chat_prompts[0])

System: Act as an expert in real estate and provide brief answers
Human: what is your name?
AI: my name is AIBot
Human: what is your name?


In [29]:
print(chat_prompts[1])

System: Act as an expert in real estate and provide brief answers
Human: what is your name?
AI: my name is AIBot
Human: explain commercial real estate to me


In [30]:
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('-----')

AI: I am AIBot, your real estate expert. How can I assist you today?
-----
Commercial real estate (CRE) refers to properties used exclusively for business purposes. This includes office buildings, retail spaces, warehouses, industrial properties, and multifamily housing (like apartment complexes). CRE is typically leased to tenants who operate businesses, and it can generate income through rent and appreciation in property value. Investors often seek CRE for its potential for higher returns compared to residential real estate.
-----


In [31]:
messages = [
        ("system", "Act as an expert in real estate and provide very detailed answers with examples"),
        ("human", "what is your name?"),
        ("ai", "my name is AIBot"),
        ("human", "{user_prompt}"),
]
chat_template = ChatPromptTemplate.from_messages(messages)
text_prompts = ["what is your name?", "explain commercial real estate to me"]
chat_prompts = [chat_template.format(user_prompt=prompt) for prompt in text_prompts]
chat_prompts

['System: Act as an expert in real estate and provide very detailed answers with examples\nHuman: what is your name?\nAI: my name is AIBot\nHuman: what is your name?',
 'System: Act as an expert in real estate and provide very detailed answers with examples\nHuman: what is your name?\nAI: my name is AIBot\nHuman: explain commercial real estate to me']

In [32]:
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('-----')

AI: My name is AIBot, and I'm here to assist you with any questions you have about real estate. Whether you're looking for information on buying, selling, investing, or understanding market trends, feel free to ask!
-----
Commercial real estate (CRE) refers to properties that are used exclusively for business purposes. This sector of real estate is distinct from residential real estate, which involves properties designed for people to live in. Understanding commercial real estate involves recognizing its various types, investment strategies, market dynamics, and the factors that influence its value.

### Types of Commercial Real Estate

1. **Office Buildings**: These properties are designed for business operations and can range from single-tenant buildings to large skyscrapers. They are often categorized into:
   - **Class A**: High-quality buildings in prime locations with top amenities (e.g., One World Trade Center in New York City).
   - **Class B**: Good quality buildings that may 

#### PromptTemplate and ChatPromptTemplate supports LCEL

`PromptTemplate` and `ChatPromptTemplate` implement the [Runnable interface](https://python.langchain.com/v0.1/docs/expression_language/interface/), the basic building block of the LangChain Expression Language (LCEL). This means they support `invoke`, `ainvoke`, `stream`, `astream`, `batch`, `abatch`, `astream_log` calls.

`PromptTemplate` accepts a dictionary (of the prompt variables) and returns a `StringPromptValue`. A `ChatPromptTemplate` accepts a dictionary and returns a `ChatPromptValue`.

In [1]:
text_prompts = ["what is your name?", "explain commercial real estate to me"]
chat_prompts = [chat_template.invoke({'user_prompt' : prompt}) for prompt in text_prompts]
chat_prompts

NameError: name 'chat_template' is not defined

In [0]:
chat_prompts[1]

In [0]:
print(chat_prompts[1].to_string())

In [0]:
chat_prompts[1].to_messages()

In [0]:
responses = chatgpt.map().invoke(chat_prompts)
for response in responses:
  print(response.content)
  print('-----')